## Informações sobre o aluno:

Nomes: Artur Matos Andrade Novais e Gustavo Luiz Bueno Pereira

Matrícula: 202202431 e 202202444

# Questão 1

Você foi convidado para participar de um projeto de uma empresa BIA Technologies, uma empresa voltada para o comércio de jogos para video-games e computadores. Esse projeto conta com uma equipe de sistemas de recomendação, na qual você foi convidado a participar. Dessa forma, você irá auxiliar o desenvolvimento de um sistema de recomendação para essa empresa.

Considere o dataset fornecido pela empresa e faça uma análise básica desse dataset. Em seguida, utilize seus conhecimentos para desenvolver e avaliar três diferentes sistemas de recomendação da seguinte forma:

a) Desenvolva um sistema de recomendação utilizando filtragem colaborativa (sem realizar a redução da dimensionalidade dos dados);

b) Desenvolva um sistema de recomendação baseado em conteúdo (sem realizar a redução da dimensionalidade dos dados);

c) Desenvolva um sistema de recomendação utilizando a redução da dimensionalidade dos dados.

Forneça as métricas de avaliações para cada um dos sistemas. Conclua suas observações de forma a convencer o gestor do projeto sobre qual foi o melhor sistema de recomendação obtido até o momento.

**Entrega da resolução da prova: dia 03/12/2024 até as 23:59, via SIGAA.**

**Obs.:** A outra equipe de sistemas de recomendação da empresa BIA Technologies apresentou uma solução atingindo as métricas abaixo. Seu sistema deverá fornecer métricas melhores que as obtidas pela outra equipe (baseline):

'ndcg@5' = 0.0016818836315317604

'ndcg@10' = 0.003073280417540927

'precision@5' = 0.00016758439071103669

'precision@10' = 8.379219535551834e-05

# Conjunto de Dados

## Carregar

In [187]:
import os
import json
import gzip 
import pandas as pd


In [188]:
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Video_Games_5.json.gz

In [189]:
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Video_Games.json.gz

In [190]:
def load_file(file_name):
    data = []
    with gzip.open(file_name) as f:
        for l in f:
            data.append(json.loads(l.strip()))

    print(file_name, len(data))
    df = pd.DataFrame.from_dict(data)
    return df

reviews_df = load_file("Video_Games_5.json.gz")
metadata_df = load_file("meta_Video_Games.json.gz")

Video_Games_5.json.gz 497577
meta_Video_Games.json.gz 84819


## Descrição das colunas

- reviewerID - ID do avaliador, por exemplo, A2SUAM1J3GNN3B
- asin - ID do produto, por exemplo, 0000013714
- reviewerName - nome do avaliador
- vote - votos úteis da avaliação
- style - um dicionário dos metadados do produto, por exemplo, "Formato" é "Capa Dura"
- reviewText - texto da avaliação
- overall - classificação do produto
- summary - resumo da avaliação
- unixReviewTime - tempo da avaliação (tempo Unix)
- reviewTime - tempo da avaliação (formato original)
- image - imagens que os usuários postam após receberem o produto

In [97]:
reviews_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style
0,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,0,NaN
1,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,0,NaN
2,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,0,NaN
3,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,0,NaN
4,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,0,NaN


- asin - ID do produto, por exemplo, 0000031852
- title - nome do produto
- feature - características do produto em formato de lista de itens
- description - descrição do produto
- price - preço em dólares americanos (no momento da coleta de dados)
- imageURL - URL da imagem do produto
- imageURLHighRes - URL da imagem do produto em alta resolução
- related - produtos relacionados (também comprados, também visualizados comprados juntos, comprar após visualizar)
- salesRank - informações do ranking de vendas
- brand - nome da marca
- categories - lista de categorias às quais o produto pertence
- tech1 - primeira tabela de detalhes técnicos do produto
- tech2 - segunda tabela de detalhes técnicos do produto
- similar - tabela de produtos similares

In [7]:
metadata_df.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


Note que o dataset é muito grande e certas manipulações podem não caber na memória RAM. Além disso, alguns desses dados podem ser de baixa qualidade. Esses dados podem ser filtrados e você pode considerar apenas os dados de maior qualidade. Alguns filtros básicos:

## **Análise e preparação dos dados (0,5 pontos)**

### Reviews

In [8]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497577 entries, 0 to 497576
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         497577 non-null  float64
 1   verified        497577 non-null  bool   
 2   reviewTime      497577 non-null  object 
 3   reviewerID      497577 non-null  object 
 4   asin            497577 non-null  object 
 5   reviewerName    497501 non-null  object 
 6   reviewText      497419 non-null  object 
 7   summary         497468 non-null  object 
 8   unixReviewTime  497577 non-null  int64  
 9   vote            107793 non-null  object 
 10  style           289237 non-null  object 
 11  image           3634 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 42.2+ MB


In [73]:
reviews_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


In [33]:
reviews_df.iloc[:, :-2].nunique()

overall                5
verified               2
reviewTime          6847
reviewerID         55223
asin               17408
reviewerName       45943
reviewText        401498
summary           284302
unixReviewTime      6847
vote                 449
dtype: int64

In [37]:
# AQUI VEMOS QUE NO DATASET DE REVIEWS, UM REVIEWER PODE AVALIAR O MESMO PRODUTO MAIS DE UMA VEZ
# Podemos fazer uma média posteriormente, para ter uma real noção do que o reviewer achou do produto
reviews_df[reviews_df[['reviewerID', 'asin']].duplicated(keep=False)]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
968,5.0,False,"05 3, 2007",A1CDNTB7377YH2,B00000DMAQ,Michael A. Newman,I really love this game that is quite similar ...,What a great game!,1178150400,NaN,NaN,NaN
969,5.0,False,"05 3, 2007",A1CDNTB7377YH2,B00000DMAQ,Michael A. Newman,I really love this game that is quite similar ...,A Whole Lot of Fun,1178150400,NaN,NaN,NaN
2407,5.0,False,"01 4, 2012",AROWZGGO4VTJU,B00000INR2,Cosmic Soujourner,I have used third party products for everythin...,Stick With First Party Hardware,1325635200,3,NaN,NaN
2408,1.0,False,"01 4, 2012",AROWZGGO4VTJU,B00000INR2,Cosmic Soujourner,I have used many third party expansion packs a...,Beware Of Third Party Expansion Paks,1325635200,NaN,NaN,NaN
2409,1.0,False,"01 4, 2012",AROWZGGO4VTJU,B00000INR2,Cosmic Soujourner,I have used a lot of third party expansion pac...,Buy Only Nintendo Brand Expansion Pak,1325635200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
495475,5.0,True,"08 13, 2018",A8J2YHESWGF2E,B014P7QI6I,Amazoncustomer,I use mine on a Raspberry Pi 3 Model B and it ...,Works on Raspberry Pi and PC,1534118400,NaN,{'Color:': ' Black17'},NaN
495476,5.0,True,"08 13, 2018",A8J2YHESWGF2E,B014P7QI6I,Amazoncustomer,"This is a great product, my son wanted a contr...",Great Product,1534118400,NaN,{'Color:': ' Black16'},NaN
495477,5.0,True,"08 13, 2018",A8J2YHESWGF2E,B014P7QI6I,Amazoncustomer,my son informs me this is a good controller lo...,Five Stars,1534118400,NaN,{'Color:': ' Black15'},NaN
495478,5.0,True,"08 13, 2018",A8J2YHESWGF2E,B014P7QI6I,Amazoncustomer,"Been looking at Steam Controllers, but they're...",Awesome little controller!,1534118400,NaN,{'Color:': ' Black14'},NaN


Começando as modificações

In [193]:
reviews_df_filtrado = reviews_df.copy()

In [ ]:
# Pode ser útil para algum cáculo futuo de importância/relevância de um review
# Tem muitos valores nulos, mas na prática, seguindo a descrição fornecida, os valores
# NAN na verdade são 0, indicando que ninguém votou o review como útil.
reviews_df_filtrado.fillna({'vote': 0}, inplace=True)

In [ ]:
# A coluna 'vote' é do tipo object, mas deveria ser do tipo int. Isso acontece porque para
# números maiores que 1000, o separador de milhar é uma vírgula. Então, vamos remover as vírgulas
# e converter a coluna para int.

reviews_df_filtrado.loc[:, 'vote'] = reviews_df_filtrado['vote'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)
reviews_df_filtrado['vote'] = reviews_df_filtrado['vote'].astype(int)

In [ ]:
# Aproveitando essa conversão de tipo, vamos converter a coluna 'overall' para int também, 
# é desnecessariamente float.
reviews_df_filtrado['overall'] = reviews_df_filtrado['overall'].astype(int)

Por já ser um número inteiro, se formos manipularmos datas, vamos utilizar unixReviewTime, e posteriormente retirar reviewTime por ser uma coluna duplicada em relação a ela. Não vamos  retirá-la agora para auxiliar na interpretação dos dados, visto que unix não é bom para isso

In [204]:
# A coluna imagem até poderia auxiliar, no entanto, menos de 1% dos produtos possuem imagem, logo
# provavelmente não conseguirá agregar o suficiente a ponto de fazer alguma diferença, principalmente em
# relação ao esforço de extrair informações dela
reviews_df_filtrado.drop(columns='image', inplace=True)

In [206]:
# Pensei que os reviewerName nulos poderiam ser preenchidos pegando seu  reviewerID, e preenchendo com base em outros registros com esse ID, na hipótese de ser apenas um erro de coleta. Mas todos os registros com os IDs desses nulos, de fato, estão com o nome nulo. No entanto, como os outros campos são válidos e seu ID também, é interessante apenas preencher como vazio para não perder esses registros.

null_names = reviews_df_filtrado[reviews_df_filtrado['reviewerName'].isnull()]['reviewerID'].unique()
print(reviews_df_filtrado[reviews_df_filtrado['reviewerID'].isin(null_names) & reviews_df_filtrado['reviewerName'].notnull()])

# Assim, vamos preencher o reviewerName com '', juntamente com os campos textuais "reviewText" e "summary", pois não há problema serem nulos.

reviews_df_filtrado.fillna({'reviewText': '', 'summary': '', 'reviewerName': ''}, inplace=True)

Empty DataFrame
Columns: [overall, verified, reviewTime, reviewerID, asin, reviewerName, reviewText, summary, unixReviewTime, vote, style]
Index: []


In [208]:
# Agora vamos retirar os registros com informações reduzidas em relaçaõ ao dataset de produtos, visto que não teremos as informações deles
reviews_df_filtrado = reviews_df_filtrado[reviews_df_filtrado['asin'].isin(metadata_df['asin'])]

In [ ]:
# Informações importantes para calcular a similaridade entre clientes, talvez seja
# interessante transformá-las em colunas categóricas
reviews_df_filtrado[reviews_df_filtrado['style'].notnull()]['style']

In [207]:
def fix_names(x):
    updated_dict = {}

    for key, value in x.items():
        new_key = key.split(':')[0]
        new_value = value.strip()
        updated_dict[new_key] = new_value

    x.clear()
    x.update(updated_dict)

    return x

reviews_df_filtrado[reviews_df_filtrado['style'].notnull()]['style'].apply(fix_names)

46                                  {'Format': 'Hardcover'}
47                                  {'Format': 'Hardcover'}
48                                  {'Format': 'Hardcover'}
49                                  {'Format': 'Hardcover'}
50                                  {'Format': 'Hardcover'}
                                ...                        
497564                                   {'Platform': 'PC'}
497565                        {'Platform': 'PlayStation 4'}
497573    {'Edition': 'Kids Room', 'Platform': 'PC Onlin...
497574    {'Edition': 'Kids Room', 'Platform': 'PC Onlin...
497575    {'Edition': 'Collection', 'Platform': 'Xbox One'}
Name: style, Length: 289237, dtype: object

In [154]:
def get_unique_keys(column):
    unique_keys = set()
    for item in column.dropna():  # Ignora os valores NaN
        if isinstance(item, dict):  # Verifica se é um dicionário
            unique_keys.update(item.keys())  # Adiciona as chaves ao set
    return unique_keys

# Aplicar na coluna
unique_keys = get_unique_keys(reviews_df_filtrado['style'])
print("Chaves únicas:", unique_keys)

Chaves únicas: {'Edition', 'Color', 'Item Package Quantity', 'Platform', 'Package Quantity', 'Format', 'Content', 'Style', 'Pattern', 'Length', 'Subscription Length', 'Platform for Display', 'Offer Type', 'Package Type', 'Configuration', 'Style Name', 'Size', 'Color Name', 'Denomination'}


### Produtos

Por enquanto não vou fazer o filtro sugerido de retirar as linhas de produtos que não estão em reviews. Pela filtragem colaborativa, de fato elas não seriam tão úteis, mas na filtragem por conteúdo, podem ser úteis, afinal, não é porque não estão nas amostras de compras, que não podem ser recomendados.

In [251]:
metadata_df_filtrado = metadata_df.copy()

In [ ]:
metadata_df_filtrado[metadata_df_filtrado.duplicated(subset='asin', keep=False)].sort_values(by='asin')

In [ ]:
# Por algum motivo, temos duplicatas no dataset de produtos, sendo
# redundância de informações, vamos removê-las.
metadata_df_filtrado.drop_duplicates(subset=['asin'], inplace=True)

In [ ]:
# A coluna "details" possui informações muito específicas do produto,
# e são poucas que estão realmente preenchidas (1968). Até tem algumas informações úteis
# como linguagem do livro, mas seria mais esforço por pouca informação. Então, vamos remove-la.
teste = metadata_df_filtrado[metadata_df_filtrado['details'] != {}]['details']
print(teste[teste.notnull()])

metadata_df_filtrado.drop(columns='details', inplace=True)


In [280]:
# As especificações técncias dos produtos estão extremamente mal extraídos, ainda em HTML
# Acredtito que retirar do HTML e organizar em uma lista seria muito trabalho para pouca informação
# relevante, então vamos remover essas colunas.

#print(metadata_df_filtrado['tech1'].unique()[1])

metadata_df_filtrado.drop(columns=['tech1', 'tech2'], inplace=True)

In [362]:
print(metadata_df_filtrado['fit'].unique())
# Fit está vazio, então vamos deletar
metadata_df_filtrado.drop(columns='fit', inplace=True)

['']


In [363]:
metadata_df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71911 entries, 0 to 84818
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      71911 non-null  object
 1   description   71911 non-null  object
 2   title         71911 non-null  object
 3   also_buy      71911 non-null  object
 4   brand         71911 non-null  object
 5   feature       71911 non-null  object
 6   rank          71911 non-null  object
 7   also_view     71911 non-null  object
 8   main_cat      71911 non-null  object
 9   similar_item  71911 non-null  object
 10  date          71911 non-null  object
 11  price         71911 non-null  object
 12  asin          71911 non-null  object
dtypes: object(13)
memory usage: 9.7+ MB


In [ ]:
# Assim como não usamos a coluna "image" no dataset de reviews, também não usaremos no dataset de produtos
metadata_df_filtrado.drop(columns=['imageURL', 'imageURLHighRes'], inplace=True)

In [ ]:
# Vou retirar a coluna "main_cat", que é "categoria principal", pois é redundante à coluna de categorias. Para esse sistema, vamos considerar todas as categorias, não só as principais
metadata_df_filtrado.drop(columns='main_cat', inplace=True) 

In [ ]:
# Como nosso objetivo é fazer um sistema de recomendação, vamos retirar as colunas que são originas de uma recumendação (also_buy e also_view)
metadata_df_filtrado.drop(columns=['also_buy', 'also_view', 'similar_item'], inplace=True)

A variável rank pode ser útil para ordenar as recomendações posteriormente

In [384]:
# A coluna de data pode até ser relevante posteriormente para fazer algum tipo de ordenação
# por data, mas não acredito que seja tão relevante para o contexto.

metadata_df_filtrado.drop(columns='date', inplace=True)

In [382]:
# Usar preço de forma relevante é uma tarefa complicada que exige um tratamento muito cuidadoso. Pode ser útil para segmentar diferentes poderes de compras: ex: um usuário que compra um fone airpod é bem diferente de um que compra um fone qcy -> um prefere produtos top de linha e tem poder aquisitivo para isso, e outro prefere produtos mais custo-benefício. O problema é que seria necessario segmentar o preço por tipo de produto, pois é claro que, por exemplo, um celular é muito mais caro que um jogo de video game. Por conta da necessidade desse tratamento mais complexo, vamos remover essa coluna.
metadata_df_filtrado.drop(columns='price', inplace=True)

In [ ]:
import re

def clean_category(x):
    text = ''
    for item in x:
        # Retirando todo tipo de resíduo de HTML
        item = re.sub(r'<[^>]+>', '', item)
        item = item.strip()
        text += item + '\n'

    return text



In [ ]:
# Analisando algumas amostras, percebi que a coluna 'feature", que adiciona alguma descrição sobre o produto, já vem junto com a coluna "category", justamente por erro de extração. Quando a coluna feature está preenchida, o registro de category vem com <span> do HTMl, e logo em baixo vem o texto da feature. Como eu achei ela consideravelmente útil, e vou utilizar uma abordagem de embedding, vou apenas limpar o resto de HTML, e utilizar a category como se estivesse concatenada com a feature. Considerando "description" como um excesso de informações desnecessárias sobre o produto, vou retirá-la.
metadata_df_filtrado.drop(columns=['description', 'feature'], inplace=True)
metadata_df_filtrado['category'] = metadata_df_filtrado['category'].apply(clean_category)

In [456]:
# Por fim, para fazer com que a coluna "brand" seja relevante, seria tão complexo quanto a coluna "price", no sentido em que teriamos que segmentar clientes por uma possibilidade de preferencia de marca. Pela pouca informação adicionada pela complexidade, vamos desconsiderá-la também.
metadata_df_filtrado.drop(columns='brand', inplace=True)

In [ ]:
# por fim, o restante do dataset:
metadata_df_filtrado.head()

,category,title,rank,asin
0,Video Games\nPC\nGames\n,Reversi Sensory Challenger,"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",0042000742
1,Video Games\nXbox 360\nGames\n\n,Medal of Honor: Warfighter - Includes Battlefi...,"[>#67,231 in Video Games (See Top 100 in Video...",0078764343
2,Video Games\nRetro Gaming & Microconsoles\nSup...,street fighter 2 II turbo super nintendo snes ...,"[>#134,433 in Video Games (See Top 100 in Vide...",0276425316
3,Video Games\nXbox 360\nAccessories\nController...,Xbox 360 MAS STICK,"[>#105,263 in Video Games (See Top 100 in Vide...",0324411812
4,Video Games\nPC\nGames\n\nGrades 2-12\nSpellin...,Phonics Alive! 3: The Speller,"[>#92,397 in Video Games (See Top 100 in Video...",0439335310


## a) (2,0 pontos) Modelo de recomendação com filtragem colaborativa**

Resolvemos utilizar uma abordagem de embeddings, ao invés de vetores esparsos como CountVectorizer ou TF-IDF.

### Encoder

In [478]:
import os, sys
import numpy as np
from numpy.typing import NDArray
from typing import List, Optional
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import vertexai
from google.cloud import aiplatform
from numpy.typing import NDArray
import numpy as np


class EmbeddingVertexAI:
    def __init__(self):
        self.model_name = 'text-multilingual-embedding-002'
        self.embedding_dim = 768
        self.embedding_model = TextEmbeddingModel.from_pretrained(self.model_name)

    @staticmethod
    def _batch_chunks(chunks: List[str], batch_size: int = 250):
        """A API tem o limite de 250 documentos por request para criar os embeddings"""
        for i in range(0, len(chunks), batch_size):
            yield chunks[i:i + batch_size]

    @staticmethod
    def _batch_inputs(inputs: List[TextEmbeddingInput], batch_size: int = 10):
        for i in range(0, len(inputs), batch_size):
            yield inputs[i:i + batch_size]

    def embed_documents(
            self, 
            chunks: List[str], 
            task: str = 'SEMANTIC_SIMILARITY',
    ) -> NDArray[np.float32]:
        """Embeds texts with a pre-trained, foundational model.
        Args:
            texts (List[str]): A list of texts to be embedded.
            task (str): The task type for embedding. Check the available tasks in the model's documentation.
            dimensionality (Optional[int]): The dimensionality of the output embeddings.
        Returns:
            List[List[float]]: A list of lists containing the embedding vectors for each input text
        """

        # A API tem o limite de 250 documentos por request para criar os embeddings
        # Então, é necessário dividir os documentos em batches
        batches: List[List[str]] = self._batch_chunks(chunks)
        all_embeddings = []
        for batch in batches:
            # Otimiza os embeddings conforme o tipo de task, melhorando, 
            # segundo a documentação, a qualidade dos embeddings
            inputs = [TextEmbeddingInput(chunk, task) for chunk in batch]
            kwargs = dict(output_dimensionality=self.embedding_dim) if self.embedding_dim else {}
            
            for inputs_batch in self._batch_inputs(inputs):
                embeddings = self.embedding_model.get_embeddings(inputs_batch, **kwargs)
                embeddings = np.array([embedding.values for embedding in embeddings], dtype=np.float32)
                all_embeddings.append(embeddings)

        embeddings = np.concatenate(all_embeddings, axis=0)

        return embeddings

## VectorStore

In [480]:
import faiss
from pandas import DataFrame
import numpy as np
from typing import Tuple
from numpy import ndarray
from typing import List
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

class VectorStoreFaissAdapter:

    def __init__(
            self, 
            encoder, 
            index_path: str = None,
            index_type: str = 'flat', 
    ):
        if index_path:
            self.index_path = index_path
        else:
            os.makedirs(os.path.join('data', 'vectorstore'), exist_ok=True)
            self.index_path = os.path.join('data', 'vectorstore')

        self.encoder = encoder
        self.vector_store = None
        self.index_type = index_type
        self.index_file = os.path.join(self.index_path, 'index.faiss')
        self.metadata_file = os.path.join(self.index_path, 'metadata.npy')  # Para armazenar títulos ou IDs
        
        if os.path.exists(self.index_file):
            self.index = self._read_index()
            self.metadata = np.load(self.metadata_file, allow_pickle=True).tolist()  # Carrega os metadados
        else:
            self.index = self._create_index(self.encoder.embedding_dim)
            self.metadata = []  # Lista para armazenar títulos ou IDs dos produtos

    def _create_index(self, embedding_dim: int):
        os.makedirs(self.index_path, exist_ok=True)

        if self.index_type == 'flat':
            return faiss.IndexFlatIP(embedding_dim)
        else:
            raise ValueError(f"Unknown index type: {self.index_type}")
        
    def _read_index(self):
        return faiss.read_index(self.index_file)
        
    def _save_index(self):
        faiss.write_index(self.index, self.index_file)
        np.save(self.metadata_file, self.metadata)  # Salva os metadados

    def add_products(self, chunks, product_titles: List[str]):
        # Gera embeddings a partir dos títulos dos produtos
        embeddings = self.encoder.embed_documents(chunks, task='SEMANTIC_SIMILARITY')
        embeddings = np.ascontiguousarray(embeddings)
        faiss.normalize_L2(embeddings)

        self.index.add(embeddings)
        self.metadata.extend(product_titles)  # Salva os títulos dos produtos
        self._save_index()

    def get_embedding_by_title(self, title: str) -> np.ndarray:
        # Busca o índice do título no metadata
        try:
            product_index = self.metadata.index(title)
            embedding = self.index.reconstruct(product_index)  # Recupera o embedding do FAISS
            return embedding
        except ValueError:
            raise ValueError(f"Produto '{title}' não encontrado no metadata.")

    def search_by_embedding(self, embedding: ndarray, k: int = 5) -> List[Tuple[str, float]]:
        embedding = embedding.reshape(1, -1)
        embedding = np.ascontiguousarray(embedding)
        faiss.normalize_L2(embedding)

        distances, indexes = self.index.search(embedding, k)

        # Mapeia os índices para os títulos/metadados dos produtos
        results = [
            (self.metadata[idx], distances[0][i]) for i, idx in enumerate(indexes[0]) if idx != -1
        ]
        return results


In [ ]:
# Implemente um sistema de recomendação utilizando filtragem colaborativa

# Insira comentários no decorrer do código

- **b) (2,0 pontos) Modelo de recomendação baseado em conteúdo**

In [ ]:
# Implemente um sistema de recomendação baseado em conteúdo

# Insira comentários no decorrer do código

- **c) (2,0 pontos) Modelo de recomendação aplicando a redução da dimensionalidade**

In [ ]:
# Implemente um sistema de recomendação aplicando a redução da dimensionalidade

# Insira comentários no decorrer do código

- **(2,0 pontos) Análise das métricas**

In [ ]:
# Calcule e informe as métricas para cada um dos modelos de recomendação desenvolvidos

# Utilize gráficos e tabelas sempre que considerar interessante

- **Conclusões (1,5 pontos)**

Insira suas conclusões aqui.

Dica: Informe as conclusões obtidas. Compare os valores das métricas obtidas entre os modelos e justifique qual seria o melhor modelo a ser utilizado no momento. Foque em convencer o gestor de que os seus modelos de recomendações são bons e qual seria o melhor a ser utilizado apresentando o motivo/justificativa.